In [1]:
# import Flow's base scenario class
from flow.scenarios import Scenario
from flow.core.params import VehicleParams
from flow.core.params import TrafficLightParams


In [2]:
# define the scenario class, and inherit properties from the base scenario class
class kruispuntc(Scenario):
    pass

In [3]:
ADDITIONAL_NET_PARAMS = {
    "num_lanes": 1,
    "speed_limit": 30,
    "traffic_lights": True,
}

# specify nodes

In [4]:
   class kruispuntc(kruispuntc): 
    def specify_nodes(self, net_params):
        # specify the name and position (x,y) of each node
        nodes = [{"id": "center", "x": 0,  "y": 0},
                 {"id": "left",  "x": -40,  "y": 0},
                 {"id": "right","x": 40,  "y": 0},
                 {"id": "top",    "x": 0,  "y": 40},
                 {"id": "bottom",   "x": 0, "y": -40}]

        return nodes

# specify edges

In [5]:
class kruispuntc(kruispuntc):
    def specify_edges(self, net_params):
        edgelen = 40
        # this will let us control the number of lanes in the network
        lanes = net_params.additional_params["num_lanes"]
        # speed limit of vehicles in the network
        speed_limit = net_params.additional_params["speed_limit"]

        edges = [{"id": "BC", "numLanes": lanes, "speed": speed_limit,
                  "from": "bottom", "to": "center", "length": edgelen},
                 {"id": "TC", "numLanes": lanes, "speed": speed_limit,
                  "from": "top", "to": "center", "length": edgelen},
                 {"id": "LC", "numLanes": lanes, "speed": speed_limit,
                  "from": "left", "to": "center", "length": edgelen},
                 {"id": "RC", "numLanes": lanes, "speed": speed_limit,
                  "from": "right", "to": "center", "length": edgelen},
                 {"id": "CB", "numLanes": lanes, "speed": speed_limit,
                  "from": "center", "to": "bottom", "length": edgelen},
                 {"id": "CT", "numLanes": lanes, "speed": speed_limit,
                  "from": "center", "to": "top", "length": edgelen},
                 {"id": "CL", "numLanes": lanes, "speed": speed_limit,
                  "from": "center", "to": "left", "length": edgelen},
                 {"id": "CR", "numLanes": lanes, "speed": speed_limit,
                  "from": "center", "to": "right", "length": edgelen}]

        return edges

# specify connections

In [6]:
class kruispuntc(kruispuntc):
    def specify_connections(self, net_params):
        conn = []
        conn += [
                     {"from": "TC",
                      "to": "CB",
                      "fromLane": "0",
                      "toLane": "0"}]
        conn += [
                     {"from": "BC",
                      "to": "CT",
                      "fromLane": "0",
                      "toLane": "0"}]
        conn += [
                     {"from": "LC",
                      "to": "CR",
                      "fromLane": "0",
                      "toLane": "0"}]
        conn += [
                     {"from": "RC",
                      "to": "CL",
                      "fromLane": "0",
                      "toLane": "0"}]
        return conn

# specify routes

In [7]:
 class kruispuntc(kruispuntc):
    def specify_routes(self, net_params):
        rts = {"TC": ["TC", "CB"],
               "BC": ["BC", "CT"],
               "LC": ["LC", "CR"],
               "RC": ["RC", "CL"],
               "CB": ["CB"],
               "CT": ["CT"],
               "CR": ["CR"],
               "CL": ["CL"]}
        return rts

In [8]:
 class kruispuntc(kruispuntc):
    def specify_edge_starts(self):

        edgestarts = [("TC", 0),
                      ("CB", 40),("CL", 40),("CR", 40)]
        
        return edgestarts

# traffic lights

In [9]:
tl_logic = TrafficLightParams()
nodes = ["center"]
phases = [{"duration": "2", "state": "GGGrrrGGGrrr"},
          {"duration": "2", "state": "yyyrrryyyrrr"},
          {"duration": "2", "state": "rrrGGGrrrGGG"},
          {"duration": "2", "state": "rrryyyrrryyy"}]
for node_id in nodes:
    tl_logic.add(node_id, tls_type="static", programID="1", offset=None, phases=phases)

# Testing the cenario

In [10]:
from flow.core.params import VehicleParams
from flow.controllers import IDMController, GridRouter
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams

vehicles = VehicleParams()
vehicles.add(veh_id="human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(GridRouter, {}),
             num_vehicles=2)
sumo_params = SumoParams(sim_step=0.1, render=True)

initial_config = InitialConfig(bunching=40)

In [11]:
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

In [12]:
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

In [13]:
from flow.core.experiment import Experiment

scenario = kruispuntc(  # we use the newly defined scenario class
    name="kruispunt-test",
    vehicles=vehicles,
    net_params=net_params,
    initial_config=initial_config,
    traffic_lights=tl_logic
)

# AccelEnv allows us to test any newly generated scenario quickly
env = AccelEnv(env_params, sumo_params, scenario)

exp = Experiment(env)

# run the sumo simulation for a set number of time steps
_ = exp.run(1, 8000)

Error during start: Traceback (most recent call last):
  File "/home/timo/BAP/Github/flow/flow/core/kernel/simulation/traci.py", line 154, in start_simulation
    traci_connection.setOrder(0)
  File "/home/timo/sumo/tools/traci/connection.py", line 300, in setOrder
    self._sendExact()
  File "/home/timo/sumo/tools/traci/connection.py", line 97, in _sendExact
    raise FatalTraCIError("connection closed by SUMO")
traci.exceptions.FatalTraCIError: connection closed by SUMO



KeyboardInterrupt: 